In [1]:
"""Mini projet sur les données de valeurs foncières en France"""
################################################################################
# fichier  : pretraitement.py
# Auteur : Rakotoson Loïc
################################################################################

################################################################################
# Importation de fonctions externes :
import numpy as np
import pickle
import csv
from os import listdir
from datetime import datetime

################################################################################
# Definition locale de fonctions :

In [2]:
# Récupération des variables
def readData(nomFichier):
    listeData = []
    try:
        with open(nomFichier, 'rb') as objet_fichier:
            objet_fichier_depickler = pickle.Unpickler(objet_fichier)
            while 1:
                try:
                    listeData.append(objet_fichier_depickler.load())
                except EOFError:
                    break
    except (IOError, OSError):
        print("Problème à l'ouverture du fichier")
    return listeData

In [3]:
# Importation des données
def lectureArray(path, filestart, listeannee, ncol=43):
    """
    filestart: nom commun des fichiers
    :return: array
    """
    i = len(filestart)
    listfiles = [
        path + files for files in listdir(path)
        if (files[:i] == filestart and files[i + 1:i + 5] in listeannee)
    ]
    header = np.genfromtxt(listfiles[0], delimiter="|", dtype=str,
                           max_rows=1).reshape((1, ncol))
    data = np.concatenate([
        np.genfromtxt(file, delimiter="|", dtype=str, skip_header=True)
        for file in listfiles
    ])
    data = np.concatenate((header, data), axis=0)
    return data

In [4]:
################################################################################
# Definition de classe

################################################################################
#Récupération des données

reponses = readData('donnees/reponses.pic')
#La variable reponses représente l'ensemble des réponses aux questions du projet
Aide_ex14 = reponses[0]
Aide_ex21 = reponses[1]
Aide_ex22 = reponses[2]
Aide_ex32 = reponses[3]
################################################################################
# Corps principal du programme :

# Pretraitement
## Exercice 1 : Lecture des données de valeurs foncières
Faire une lecture d’un fichier de valeurs foncières (« valeursfoncieres-20XX_rennes_metropole.txt»). Le format du fichier est de type texte et le séparateur de champ est le symbole «| » (ce fichier peut-être vu comme un fichier « csv »)

In [5]:
data = lectureArray("donnees/", "valeursfoncieres",
                    ("2015", "2016", "2017", "2018"))

## Exercice 2 : Lecture des données cadastrales des parcelles
Le rôle de ce fichier est de situer chaque bien par sa parcelle. Chaque bien des fichiers de valeursfoncières est identifié par ses références cadastrales (« Code departement », « Code commune », «Prefixe de section », « Section », « No plan »).

Faire une lecture du fichier de données cadastrales en faisant en sorte d’obtenir un dictionnairedont la clé correspondra aux références cadastrales (clé : (commune, préfixe, section, numéro) ;Attention dans ce fichier « Code departement » et « Code commune » sont regroupés sous le nom « commune ») et la valeur correspondra aux coordonnées géographiques de la parcelle.

In [7]:
with open("donnees/cadastre.csv", 'r', encoding="utf-8") as file:
    cadastre = [ligne for ligne in csv.reader(file, delimiter=";")]

elt = [
    cadastre[0].index(c)
    for c in ("commune", "prefixe", "section", "numero", "coordinates")
]
cadastre = {(row[elt[0]], row[elt[1]], row[elt[2]], row[elt[3]]): row[elt[4]]
            for row in cadastre[1:]}